### Notebook to parse scraped html to produce cleaned text of CHRT decisions

Author: Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/). NOTE: Users must also comply with upstream [licensing](https://www.chrt-tcdp.gc.ca/transparency/terms-and-conditions-en.html) for the CHRT data source, as well as requests on source urls not to allow indexing of the documents by search engines to protect privacy. As a result, users must not make the data available in formats or locations that can be indexed by search engines.

Dataset & Code to be cited as: 

    Sean Rehaag, "Canadian Human Rights Tribunal Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/chrt>.

To load data, see [load_and_analyze_chrt_cases.ipynb](https://github.com/Refugee-Law-Lab/chrt_bulk_data/blob/master/load_and_analyze_chrt_cases.ipynb)

### Notes:

(1) Data Source: [Canadian Human Rights Tribunal](https://www.chrt-tcdp.gc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Canadian Human Rights Tribunal website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Canadian Human Rights Tribunal.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Canadian Human Rights Tribunal website's [Terms of Use](https://www.chrt-tcdp.gc.ca/transparency/terms-and-conditions-en.html).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. e license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Supreme Court of Canada website's [Terms of Use](https://www.scc-csc.ca/terms-avis/notice-enonce-eng.aspx).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 



In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/CHRT/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/chrt_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/chrt_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/chrt_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')
out_path_HF = pathlib.Path('d:/AI-Projects/canadian-legal-data/CHRT/train.parquet')

# set years sought
start_year = 2003
end_year = 2024

# set language sought
language_sought = None  #set to 'en' if english only, set to 'fr' if French only, set to None if all languages


### Load Raw Data

In [2]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        results.append(data)

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)

#df


1880


100%|██████████| 1880/1880 [00:10<00:00, 178.39it/s]


In [3]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [4]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# filter for langauge if desired
if language_sought:
    df = df[df.case_language == language_sought]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]

# if citation2 = ciation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [5]:
# Run this in parallel to speed up

import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

# Convert the pandas DataFrame to a Dask DataFrame
ddf = dd.from_pandas(df, npartitions=16) 

# Apply the function in parallel
ddf['text'] = ddf.html.apply(get_text)

# Convert the Dask DataFrame back to a pandas DataFrame
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        df = ddf.compute()


[                                        ] | 0% Completed | 124.71 ms

d:\AI-Projects\venv311\Lib\site-packages\dask\dataframe\core.py:4378: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('html', 'object'))

  warnings.warn(meta_warning(meta))


[########################################] | 100% Completed | 14.71 s


In [6]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 1877/1877 [00:04<00:00, 442.00it/s]


In [7]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

# rename decision_date to document_date
df = df.rename(columns={'decision_date': 'document_date'})

# add dataset column
df['dataset'] = 'CHRT'

# add other column
df['other'] = ''

# reorder columns
list_cols = ['citation',
             'citation2', 
             'dataset', 
             'year', 
             'name', 
             'language', 
             'document_date', 
             'source_url', 
             'scraped_timestamp', 
             'unofficial_text',
             'other']

df = df[list_cols]

# reindex
df = df.reset_index(drop=True)

In [8]:
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2003 CHRT 1,,CHRT,2003,"Communications, Energy and paperworkers union ...",en,2003-01-10,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2023-12-01,"Communications, Energy and paperworkers union ...",
1,2003 CHRT 10,,CHRT,2003,Parisien v. Ottawa-Carleton Regional Transit,en,2003-03-06,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2023-12-01,Parisien v. Ottawa-Carleton Regional Transit\n...,
2,2003 CHRT 11,,CHRT,2003,Hodgins v. Transport North American Express Inc.,en,2003-03-06,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2023-12-01,Hodgins v. Transport North American Express In...,
3,2003 CHRT 12,,CHRT,2003,Day v. Department of National Defence and Mich...,en,2003-03-07,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2023-12-01,Day v. Department of National Defence and Mich...,
4,2003 CHRT 13,,CHRT,2003,Day v. Canada (Department of National Defence),en,2003-03-12,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2023-12-01,Day v. Canada (Department of National Defence)...,


In [9]:
df.tail()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
1872,2024 TCDP 93,,CHRT,2024,Heddle c. Société canadienne des postes,fr,2024-07-31,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2024-09-08,Heddle c. Société canadienne des postes\nColle...,
1873,2024 TCDP 94,,CHRT,2024,Miller c. Banque Toronto-Dominion,fr,2024-08-01,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2024-09-08,Miller c. Banque Toronto-Dominion\nCollection\...,
1874,2024 TCDP 95,,CHRT,2024,Société de soutien à l’enfance et à la famille...,fr,2024-08-02,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2024-09-08,Société de soutien à l’enfance et à la famille...,
1875,2024 TCDP 96,,CHRT,2024,Société de soutien à l’enfance et à la famille...,fr,2024-08-09,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2024-09-08,Société de soutien à l’enfance et à la famille...,
1876,2024 TCDP 97,,CHRT,2024,Sewap c. Service correctionnel du Canada,fr,2024-08-23,https://decisions.chrt-tcdp.gc.ca/chrt-tcdp/de...,2024-09-08,Sewap c. Service correctionnel du Canada\nColl...,


In [10]:
# count number of cases per year
df.groupby('year').count()

,citation,citation2,dataset,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
year,,,,,,,,,,
2003,90,90,90,90,90,90,90,90,90,90
2004,78,78,78,78,78,78,78,78,78,78
2005,92,92,92,92,92,92,92,92,92,92
2006,115,115,115,115,115,115,115,115,115,115
2007,112,112,112,112,112,112,112,112,112,112
2008,100,100,100,100,100,100,100,100,100,100
2009,73,73,73,73,73,73,73,73,73,73
2010,68,68,68,68,68,68,68,68,68,68
2011,46,46,46,46,46,46,46,46,46,46


### Export data

In [11]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [12]:
# export cleaned df to parquet

df.to_parquet(out_path_parquet)

In [13]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    df[df.year == year].to_json(out_path_yearly / f'{year}.json', orient='records', indent=4)

100%|██████████| 22/22 [00:00<00:00, 28.53it/s]


In [14]:
# export to huggingface format
df.to_parquet(out_path_HF)